In [52]:
#import dependencies
import pandas as pd
import os

In [53]:
#set file_path for items_complete.csv
hawaii_measurements_csv = os.path.join("Instructions", "Resources","clean_hawaii_measurements.csv")
hawaii_stations_csv = os.path.join("Instructions", "Resources","clean_hawaii_stations.csv")

#read in text
hawaii_measurements = pd.read_csv(hawaii_measurements_csv, dtype=object)
hawaii_stations = pd.read_csv(hawaii_stations_csv, dtype=object)


In [54]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, MetaData

In [55]:
# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base

In [56]:
from sqlalchemy import Column, Integer, String, Float

# Create database using engine and call it hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

#Create a connection to the engine
conn = engine.connect()

In [57]:
# Use `declarative_base` from SQLAlchemy to model the Measurement table as an ORM class
Base = declarative_base()

#Create class for Measurement
class Measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)

    def __repr__(self):
        return f"id={self.id}, name={self.name}"    
        
#Create class for Station
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float) 
    elevation = Column(Float) 
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [58]:
#Create the tables in the database
Base.metadata.create_all(engine)

In [59]:
#Use Orient='records' to create a list of data to write
measurement_data = hawaii_measurements.to_dict(orient='records')
station_data = hawaii_stations.to_dict(orient='records')

In [60]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

# Save the reference to the `measurement` and 'station' table as a variable
measurement = sqlalchemy.Table('measurement', metadata, autoload=True)
station = sqlalchemy.Table('station', metadata, autoload=True)

In [61]:
conn.execute(measurement.delete())
conn.execute(station.delete())

In [62]:
#Insert the measurement data in dictionary format into the sqlite
conn.execute(measurement.insert(), measurement_data)

#Insert the stations data in dictionary format into the sqlite
conn.execute(station.insert(), station_data)

In [63]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

In [64]:
#!rm hawaii.sqlite